In [11]:
import pandas as pd

# Charger le fichier CSV
df = pd.read_csv('raw.csv')
df.head()

,Start,Diver,Gender,Discipline,Line,Official Top,AP,RP,Card,Points,Remarks,Title Event,Event Type,Day,Category Event
0,1,Tasos Grillakis (GRC),M,FIM,NaN,00:00,33,23 m,YELLOW,12.0,-,Depth Event 2016,Depth Competition,2016-07-17,other
1,2,Antonis Papantonatos (GRC),M,FIM,NaN,00:00,55,47 m,YELLOW,38.0,-,Depth Event 2016,Depth Competition,2016-07-17,other
2,3,Dimitris Koumoulos (GRC),M,CNF,NaN,00:00,55,55 m,WHITE,55.0,-,Depth Event 2016,Depth Competition,2016-07-17,other
3,4,Christos Papadopoulos (GRC),M,CWT,NaN,00:00,55,55 m,WHITE,55.0,OK,Depth Event 2016,Depth Competition,2016-07-17,other
4,5,Anna Chalari (GRC),F,CWT,NaN,00:00,15,15 m,WHITE,15.0,OK,Depth Event 2016,Depth Competition,2016-07-17,other


In [12]:
def clean_and_transform_data(df):
    """Clean and transform the DataFrame by removing specific columns, converting categorical data to numerical, 
    adding new columns based on existing data, and calculating cumulative counts.

    This function performs the following operations:
    - Removes unnecessary columns ('Line' and 'Official Top').
    - Converts 'Gender' to numerical values (M=1, F=0).
    - Converts 'Discipline' to numerical values based on unique disciplines present in the data.
    - Adds a 'Month' column extracted from the 'Day' column.
    - Calculates the cumulative count of dives per diver as 'Experience Dive'.
    - Calculates the cumulative count of dives per diver per discipline as 'Experience Discipline'.

    Args:
        df (DataFrame): The DataFrame to be cleaned and transformed.
    
    Returns:
        DataFrame: The cleaned and transformed DataFrame.
    """
    import pandas as pd

    # Remove 'Line' and 'Official Top' columns
    df.drop(columns=['Line', 'Official Top'], inplace=True)

    # Convert 'Gender' to numerical values (M=1, F=0)
    gender_to_int = {'M': 1, 'F': 0}
    df['Gender'] = df['Gender'].map(gender_to_int)

    # Convert 'Discipline' to numerical values
    disciplines = df['Discipline'].unique()
    discipline_to_int = {disc: idx for idx, disc in enumerate(disciplines)}
    df['Discipline'] = df['Discipline'].map(discipline_to_int)

    # Add 'Month' column extracted from 'Day'
    df['Month'] = pd.to_datetime(df['Day']).dt.month

    # Calculate general diving experience
    df['Experience Dive'] = df.groupby('Diver').cumcount()

    # Calculate diving experience in each discipline
    df['Experience Discipline'] = df.groupby(['Diver', 'Discipline']).cumcount()

    return df


In [13]:
df=clean_and_transform_data(df)
# Afficher les premières lignes du DataFrame nettoyé
df.head()

,Start,Diver,Gender,Discipline,AP,RP,Card,Points,Remarks,Title Event,Event Type,Day,Category Event,Month,Experience Dive,Experience Discipline
0,1,Tasos Grillakis (GRC),1,0,33,23 m,YELLOW,12.0,-,Depth Event 2016,Depth Competition,2016-07-17,other,7,0,0
1,2,Antonis Papantonatos (GRC),1,0,55,47 m,YELLOW,38.0,-,Depth Event 2016,Depth Competition,2016-07-17,other,7,0,0
2,3,Dimitris Koumoulos (GRC),1,1,55,55 m,WHITE,55.0,-,Depth Event 2016,Depth Competition,2016-07-17,other,7,0,0
3,4,Christos Papadopoulos (GRC),1,2,55,55 m,WHITE,55.0,OK,Depth Event 2016,Depth Competition,2016-07-17,other,7,0,0
4,5,Anna Chalari (GRC),0,2,15,15 m,WHITE,15.0,OK,Depth Event 2016,Depth Competition,2016-07-17,other,7,0,0


In [14]:
import numpy as np

In [15]:
# Identify unexploitable values (e.g., '()' in the 'Diver' column)
# and consider them as NaN for the calculation of missing data.
df.replace({'()': np.nan}, inplace=True)

# Calculate the total percentage of missing data after replacement.
total_cells = np.product(df.shape)
total_missing = df.isnull().sum().sum()
percentage_missing = (total_missing / total_cells) * 100

# Calculate the percentage of missing data per column after replacement.
percentage_missing_per_column = (df.isnull().sum() / df.shape[0]) * 100

# Calculate the number of rows with missing data after replacement.
rows_with_missing = df.isnull().any(axis=1).sum()

# Display the information.
print(f"Total percentage of missing data after replacement: {percentage_missing:.2f}%")
print(f"Percentage of missing data per column after replacement:\n{percentage_missing_per_column}")
print(f"Number of rows with missing data after replacement: {rows_with_missing}")
print(f"Percentage of rows with missing data relative to total rows after replacement: {(rows_with_missing / df.shape[0]) * 100:.2f}%")

# You might also want to clean specific columns of undesirable formats.
# For example, removing parentheses from the 'Diver' column.
df['Diver'] = df['Diver'].str.replace(r"\(\)", "", regex=True)

Total percentage of missing data after replacement: 0.01%
Percentage of missing data per column after replacement:
Start                    0.000000
Diver                    0.096867
Gender                   0.000000
Discipline               0.000000
AP                       0.000000
RP                       0.000000
Card                     0.000000
Points                   0.000000
Remarks                  0.018628
Title Event              0.000000
Event Type               0.000000
Day                      0.000000
Category Event           0.000000
Month                    0.000000
Experience Dive          0.000000
Experience Discipline    0.000000
dtype: float64
Number of rows with missing data after replacement: 31
Percentage of rows with missing data relative to total rows after replacement: 0.12%


In [16]:
def clean_data(df):
    """Clean the DataFrame by replacing unexploitable values with NaN and removing rows with missing values.

    This function performs the following operations:
    - Replaces specified unexploitable values like '()', empty strings, and spaces with NaN.
    - Removes all rows that have any missing values (NaN).

    Args:
        df (DataFrame): The DataFrame to be cleaned.
    
    Returns:
        DataFrame: The cleaned DataFrame.
    """
    import numpy as np

    # Replace unexploitable values with NaN
    df.replace({'()': np.nan, '': np.nan, ' ': np.nan}, inplace=True)

    # Remove all rows with missing values
    df_cleaned = df.dropna()

    return df_cleaned

def save_cleaned_data(df, file_path='processed.csv'):
    """Save the cleaned DataFrame to a CSV file.

    Args:
        df (DataFrame): The DataFrame to be saved.
        file_path (str): The file path to save the DataFrame.
    """
    df.to_csv(file_path, index=False)


In [17]:
df = clean_data(df)
save_cleaned_data(df)

In [18]:
df.head()

,Start,Diver,Gender,Discipline,AP,RP,Card,Points,Remarks,Title Event,Event Type,Day,Category Event,Month,Experience Dive,Experience Discipline
0,1,Tasos Grillakis (GRC),1,0,33,23 m,YELLOW,12.0,-,Depth Event 2016,Depth Competition,2016-07-17,other,7,0,0
1,2,Antonis Papantonatos (GRC),1,0,55,47 m,YELLOW,38.0,-,Depth Event 2016,Depth Competition,2016-07-17,other,7,0,0
2,3,Dimitris Koumoulos (GRC),1,1,55,55 m,WHITE,55.0,-,Depth Event 2016,Depth Competition,2016-07-17,other,7,0,0
3,4,Christos Papadopoulos (GRC),1,2,55,55 m,WHITE,55.0,OK,Depth Event 2016,Depth Competition,2016-07-17,other,7,0,0
4,5,Anna Chalari (GRC),0,2,15,15 m,WHITE,15.0,OK,Depth Event 2016,Depth Competition,2016-07-17,other,7,0,0
